In [ ]:
# %%
# filename: optim_walk_forward_analysis_sma.ipynb

# --- 1. Imports and Setup ---
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import EURUSD
import pandas_ta as ta

# --- Matplotlib Setup for Pandas Plots ---
import matplotlib.pyplot as plt
%matplotlib inline
# This magic command tells Jupyter to render plots directly in the output cell.
plt.style.use('dark_background') # Optional: Set a dark theme consistent with other plots

# --- 2. Load Sample Data ---
print("Loading built-in EURUSD sample data...")
data = EURUSD.copy()
print(f"Data loaded. Contains {len(data)} bars from {data.index[0]} to {data.index[-1]}.")

In [ ]:
# %%
# --- 3. The Parameterized Strategy Class ---
# This is the same strategy we optimized in the previous notebook.

class SmaCrossSLTP(Strategy):
    fast_period = 10
    slow_period = 40
    sl_percent = 1.5
    tp_percent = 3.0

    def init(self):
        self.fast_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.fast_period)
        self.slow_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.slow_period)

    def next(self):
        sl_price = self.data.Close[-1] * (1 - self.sl_percent / 100)
        tp_price = self.data.Close[-1] * (1 + self.tp_percent / 100)

        if crossover(self.fast_sma, self.slow_sma):
            if not self.position:
                self.buy(sl=sl_price, tp=tp_price)
        elif crossover(self.slow_sma, self.fast_sma):
            self.position.close()

In [ ]:
# %%
# --- 4. Walk-Forward Analysis Implementation ---
print("\n--- Starting Walk-Forward Analysis ---")

# --- WFA Configuration ---
# Using integer indices for slicing the DataFrame
total_bars = len(data)
# ~6 months of training data, ~2 months of validation data (4-hour bars)
n_train_bars = int(total_bars * 0.25) # Use 25% of data for the first training
n_test_bars = int(n_train_bars / 3)   # Test period is 1/3 of the training period
step = n_test_bars                    # Slide the window forward by the test period size

# List to store the results from each out-of-sample (OOS) run
oos_results = []

# The main loop for walking forward
for i in range(n_train_bars, total_bars - n_test_bars, step):
    # Define the training and testing data slices
    train_data = data.iloc[i - n_train_bars : i]
    test_data = data.iloc[i : i + n_test_bars]

    print("-" * 50)
    print(f"Training on: {train_data.index[0]} -> {train_data.index[-1]} ({len(train_data)} bars)")
    print(f"Testing on:  {test_data.index[0]} -> {test_data.index[-1]} ({len(test_data)} bars)")

    # --- In-Sample Optimization ---
    bt_train = Backtest(train_data, SmaCrossSLTP, cash=10000, commission=.002, finalize_trades=True)

    # Using a smaller parameter grid for faster WFA execution
    stats_train, _ = bt_train.optimize(
        fast_period=[10, 20],
        slow_period=[40, 50],
        sl_percent=[1.0, 1.5],
        tp_percent=[2.0, 3.0],
        maximize='SQN',
        constraint=lambda p: p.fast_period < p.slow_period,
        return_heatmap=True
    )

    best_params = stats_train._strategy
    print(f"Best params found in-sample: fast={best_params.fast_period}, slow={best_params.slow_period}, sl={best_params.sl_percent}%, tp={best_params.tp_percent}%")

    # --- Out-of-Sample Validation ---
    bt_test = Backtest(test_data, SmaCrossSLTP, cash=10000, commission=.002, finalize_trades=True)

    stats_test = bt_test.run(
        fast_period=best_params.fast_period,
        slow_period=best_params.slow_period,
        sl_percent=best_params.sl_percent,
        tp_percent=best_params.tp_percent
    )

    oos_results.append(stats_test)

In [ ]:
# %%
# --- 5. Analyze WFA Results ---
print("\n\n" + "="*50)
print("--- Walk-Forward Analysis Final Results ---")

if oos_results:
    # Create a DataFrame from the list of Series results
    wfa_df = pd.concat([res.to_frame(f"Period_{i}") for i, res in enumerate(oos_results)], axis=1).T

    print("\nPerformance of each Out-of-Sample period:")
    print(wfa_df[['Return [%]', 'Sharpe Ratio', 'Win Rate [%]', '# Trades', 'Profit Factor']])

    print("\nAverage performance across all OOS periods:")
    print(wfa_df[['Return [%]', 'Sharpe Ratio', 'Win Rate [%]']].mean())

    # Plot the equity curve of each OOS period
    print("\nPlotting cumulative equity of OOS periods...")
    # This plot will now render correctly thanks to the setup in the first cell
    wfa_df['Equity Final [$]'].plot(
        kind='bar',
        title='Final Equity per Out-of-Sample Period',
        xlabel='WFA Period',
        ylabel='Final Equity [$]',
        figsize=(10, 6), # Optional: make the plot bigger
        grid=True
    )
    plt.show() # Explicitly showing the plot is the most robust way
else:
    print("No out-of-sample periods were successfully tested.")